In [13]:
from datetime import datetime,date,timedelta
from dateutil.relativedelta import relativedelta
import psycopg2
import os
import pandas as pd
import numpy as np
import xlrd
import datetime as dt

import re
from functools import reduce

In [14]:
import sys
#sys.path.insert(1, r'C:\Users\Ilyasov\Downloads\Python_prj\GitProjects\local_python_scripts\function_library'.replace('\\',"/"))
sys.path.insert(1, r'C:\Users\mendgaziev\Desktop\Git\local_python_scripts\function_library'.replace('\\',"/"))

from CDU_library import Trigram

def sim(series, text):
    row = series.apply(lambda x : Trigram().similarity(x, text)).sort_values(ascending = False)
    val = row.values[0]
    idx = row.index[0]
    return series[idx], idx, val

## Выбор параметров

In [15]:
with_test = True

In [16]:
airflow = False

In [17]:
path_airflow = ''

In [18]:
#path_git = 'M:/toplivo/408/РЕЗЕРВУАРЫ/3-PS-2023/'
#path_git = r"""C:\Users\Ilyasov\Downloads\Python_prj\GitProjects\local_python_scripts\3ps/""".replace('\\',"/")
path_git = r"""C:\Users\mendgaziev\Desktop\Git\3ps for insert/""".replace('\\', "/") 

In [27]:
# разница в месяце от текущей даты
diff_in_time = 7

## Function

In [29]:
months = {'январь' : '01',
        'февраль' : '02',
        'март' : '03',
        'апрель' : '04',
        'май' : '05',
        'июнь' : '06',
        'июль' : '07',
        'август' : '08',
        'сентябрь' : '09',
        'октябрь' : '10',
        'ноябрь' : '11',
        'декабрь' : '12'}

conn = psycopg2.connect(dbname='', user='', password='', host='',port=5432)
cur = conn.cursor()

query_3ps_company = """select id, name, okpo from _3ps_company"""
_3ps_company = pd.read_sql(query_3ps_company, conn)
_3ps_company = _3ps_company.astype(str)

query_3ps_products = """select * from _3ps_products"""
_3ps_products = pd.read_sql(query_3ps_products, conn)
_3ps_products = _3ps_products.astype(str)

query_3ps_producer = """select id, name, okpo, type from _3ps_producer"""
_3ps_producer = pd.read_sql(query_3ps_producer, conn)
_3ps_producer = _3ps_producer.astype(str)

query_3ps_transport_type = "select id, name from _3ps_transport_type"
_3ps_transport_type = pd.read_sql(query_3ps_transport_type, conn)
_3ps_transport_type = _3ps_transport_type.astype(str)


def clean_text(df, *columns):
    for column in columns:
        df[column] = df[column].astype(str)
        df[column] = df[column].replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ").replace("\n", " ").str.strip()
        if ((column == 'code_okpd2') | (column == 'okpo')):
            df[column] = df[column].replace(" ", "")
            
    return df


err_files = []          #список с файлами в которых ошибки
errors = []             #список со всеми ошибками

def err_file(file_name,err_str):
    if file_name not in err_files:
        err_files.append(file_name)
    if file_name+" - "+err_str not in errors:
        errors.append(file_name+" - "+err_str+"\n")
        
def find_id_company(df, company_name, company_okpo):
    delete = ['"', ' ', 'ооо', 'оао', 'пао', 'ао', 'зао']
    
    for str_to_del in delete:
        df['name'] = df['name'].str.lower().str.strip().str.replace(str_to_del, '')
        _3ps_df = df[ (df['name'] == company_name.lower().replace(str_to_del, '').strip())
                    | (df['okpo'].str.replace(" ", "").str.strip() == company_okpo.replace(" ", "").strip())]
    if _3ps_df['id'].shape[0] > 0:
        return str(_3ps_df['id'].values[0])
    return err_file(file_name, 'не найден id компании')   

def find_id_product(df, product, okpd2):
    _3ps_df = df[ (df.code_okpd2.str.replace(" ", "").str.strip() == okpd2.replace(" ", "").strip()) |
                  (df.name.str.lower().str.strip() == product.lower().strip()) ]
    if _3ps_df.shape[0] > 0:
        return _3ps_df.id.values[0]
    return err_file(file_name, product + ' - '+ okpd2 + ' - не найден id продукта')
                
def find_id_producer(df, name, okpo):   
    delete = ['"', ' ', 'ооо', 'оао', 'пао', 'ао', 'зао']
    for str_to_del in delete:
        df['name'] = df['name'].str.lower().str.strip().str.replace(str_to_del, '')
        _3ps_df = df[ (df.name == name.lower().replace(str_to_del, '').strip())
                 | (df.okpo.str.replace(" ", "").str.strip() == okpo.replace(" ", "").strip())    ]
    if _3ps_df.shape[0] > 0:
        return _3ps_df.id.values[0] 
    return 'Null'

def border_transport(border):
    if border == 'nan':
        return 'Null'
    else:
        return border
    
def find_id_transport_type(text):
    _3ps_df = _3ps_transport_type[   _3ps_transport_type.name.str.lower().str.strip() == text.lower().strip()    ]
    if _3ps_df.shape[0] == 0:
        return 'Null'
    return _3ps_df.id.values[0]

def max_id(txt):
    query_id = f"select max(id) from { txt }"
    max_id = pd.read_sql(query_id, conn)
    max_id = max_id['max'].max()
    
    if pd.isna(max_id):
        max_id = 0
    return max_id

def insert_vals(table, vals):
    return f""" insert into {table} values {vals}; \n""".replace("'Null'", "Null").replace("nan", "Null")

C:\Users\mendgaziev\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\mendgaziev\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\mendgaziev\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [30]:
if airflow:
    path = path_airflow
else:
    path = path_git
    
# получение нужного месяца
curr_date_prev_month = date.today() - relativedelta(months = diff_in_time)
first_date_prev_month = date(curr_date_prev_month.year, curr_date_prev_month.month, 1)
report_period_in_path = curr_date_prev_month.strftime('%m-%Y').replace('20', '')

path += report_period_in_path + "/"

files = os.listdir(path)
extensions = ('.xlsx', '.xlsb', '.xlsm', '.xls')
file_excel = [f for f in files if f.endswith(extensions)]

insert_query_entrance = ''
entrance_table_name = "_3psold_entrance"
entrance_maxid = max_id(entrance_table_name)

insert_query_shipment = ''
shipment_table_name = "_3psold_shipment"
shipment_maxid = max_id(shipment_table_name)

insert_query_balance = ''
balance_table_name = "_3psold_balance"
balance_maxid = max_id(balance_table_name)


for file_name in file_excel:
    if with_test:
        print(file_name)
    if file_name.startswith('~$'):
        continue
    
    try:
        df_shapka = pd.read_excel(path + file_name, sheet_name = 'Шапка', dtype=str)
        df_shapka = df_shapka.astype(str)
        df_entrance = pd.read_excel(path + file_name, sheet_name = '1. Производство', header=2, dtype=str).iloc[1:, 1:4]
        df_entrance = df_entrance.astype(str)
        df_shipment = pd.read_excel(path + file_name, sheet_name = '2. Отгрузка', header=2, dtype=str).iloc[1:, 1:12]
        df_shipment = df_shipment.astype(str)
        df_balance = pd.read_excel(path + file_name, sheet_name = '3. Остатки', header=2, dtype=str).iloc[1:, 1:8]
        df_balance = df_balance.astype(str)
    
    except PermissionError:
        err_file(file_name,"файл открыт был открыт на другом устройстве")
        continue
    except ValueError:
        err_file(file_name,"Новая версия формы 3пс")
        continue
    except xlrd.biffh.XLRDError:
        err_file(file_name,"Странная ошибка в файле(возможно стоит пароль)")
        continue
    else:
        reporting_month_in_file = months[df_shapka.iloc[10, 3].lower().strip()]
        reporting_year_in_file = df_shapka.iloc[10, 4].replace('20', '')
        reporting_period_in_file = reporting_month_in_file + '-' + reporting_year_in_file
               
        if reporting_period_in_file != report_period_in_path:
            err_file(file_name, 'Период отчета на титульнике не совпадает с отчетным месяцем')
            
        # связь с company_id
        #df_shapka = clean_text(df_shapka, "company_name", "company_okpo")
        company_name = df_shapka.iloc[22, 3]
        company_okpo = df_shapka.iloc[27, 2]
        company_id = find_id_company(_3ps_company, company_name, company_okpo)
        
        df_entrance.columns = ['product_name', 'code_okpd2', 'volume']   
        df_entrance['volume'] = df_entrance['volume'].replace("nan", "0")
        df_entrance = clean_text(df_entrance, "product_name", "code_okpd2")
        df_entrance = df_entrance[(df_entrance.volume != '0') & (df_entrance.volume != 'nan')]
        df_entrance = df_entrance[(df_entrance.product_name != '0') & (df_entrance.product_name != 'nan')]
       
        date_str = f"{ first_date_prev_month }"
        
        insert_query = []
        for i, row in df_entrance.reset_index(drop=True).iterrows():
            product_name = row['product_name']
            product_okpd2 = row['code_okpd2']
            product_id = find_id_product(_3ps_products, product_name, product_okpd2)
            
            
            volume = row['volume']
            
            vals = (entrance_maxid + 1 + i, company_id, product_id, volume, date_str)
                        
            insert_query.append(    insert_vals(entrance_table_name, vals)    )

        print(entrance_maxid)
        if (len(insert_query) > 0) & (file_name not in err_files):
            iq = reduce(lambda x,y: x+y, insert_query)
            insert_query_entrance += iq
            entrance_maxid += len(insert_query)
            print(iq)
        else:
            print ('Нет поступлений - ', file_name)
                  
        df_shipment.columns = ['base_name', 'product_name', 'code_okpd2', 'producer_name', 'producer_okpo',
                               'transport_type', 'reg_okato', 'country_oksm', 'border_type', 'organ_code', 'volume']
        df_shipment = clean_text(df_shipment, "product_name", "code_okpd2", "producer_name", "producer_okpo")
        df_shipment = df_shipment[(df_shipment.volume != '0') & (df_shipment.volume != 'nan')]
        df_shipment = df_shipment[(df_shipment.product_name != '0') & (df_shipment.product_name != 'nan')]

        insert_query = []
        for i, row in df_shipment.reset_index(drop=True).iterrows():    
            base_name = row['base_name']
            
            product_name = row['product_name']
            code_okpd2 = row['code_okpd2']
            product_id = find_id_product(_3ps_products, product_name, code_okpd2)
            
            producer_name = row['producer_name']
            producer_okpo = row['producer_okpo'] 
            producer_id = find_id_producer(_3ps_producer, producer_name, producer_okpo)
            
            transport_type = row['transport_type']
            transport_type = find_id_transport_type(transport_type)            
            
            reg_okato = row['reg_okato']
            country_oksm = row['country_oksm']
            
            border_type = row['border_type']
            border_type = border_transport(border_type)                      
            
            if country_oksm != 'nan':
                is_export = True
            else:
                is_export = False                
                
            volume = row['volume']
            
            vals = (shipment_maxid + 1 + i, base_name, product_id, producer_id, producer_name, producer_okpo, 
                    transport_type, reg_okato, country_oksm, border_type, is_export, volume, company_id, date_str)
            insert_query.append(insert_vals(shipment_table_name, vals)) 
        
        print(shipment_maxid)
        if (len(insert_query)>0) & (file_name not in err_files):
            iq = reduce(lambda x,y: x+y, insert_query)
            insert_query_shipment += iq
            shipment_maxid += len(insert_query)
            print(iq)
        else:
            print ('Нет отгрузок - ', file_name)
            
        df_balance.columns = ['base_name', 'product_name', 'product_code', 'producer_name', 'producer_okpo', 
                              'res_start', 'res_end']
        df_balance.res_start = df_balance.res_start.str.replace("nan", "0")
        df_balance.res_start = df_balance.res_end.str.replace("nan", "0")
        df_balance = df_balance[(df_balance.res_start != "0") & (df_balance.res_end != "0")]
        df_balance = df_balance[(df_balance.product_name != 'nan') & (df_balance.product_name != '0')]
        df_balance = clean_text(df_balance, 'product_name', 'product_code', 'producer_name', 'producer_okpo')
        
        insert_query = []
        for i, row in df_balance.reset_index(drop=True).iterrows():
            base_name = row['base_name']
            
            product_name = row['product_name']
            product_code = row['product_code']
            product_id = find_id_product(_3ps_products, product_name, product_code)
            
            producer_name = row['producer_name']
            producer_okpo = row['producer_okpo']
            producer_id = find_id_producer(_3ps_producer, producer_name, producer_okpo)
            
            res_start = row['res_start']
            res_end = row['res_end']
            
            vals = (balance_maxid + 1 + i, company_id, base_name, product_id, producer_id, producer_name, producer_okpo,
                   res_start, res_end, date_str)
            insert_query.append(insert_vals(balance_table_name, vals))
            
        print(balance_maxid)
        if (len(insert_query) > 0) & (file_name not in err_files):
            iq = reduce(lambda x,y: x+y, insert_query)
            insert_query_balance += iq
            balance_maxid += len(insert_query)
            print(iq)
        else:
            print('Нет остатков - ', file_name)
            
        # удаление дублирующих данных
        query_entr_for_del = """select * from _3psold_entrance where company_id=%s and date='%s'"""%(company_id, date_str)
        df_entr_for_del = pd.read_sql(query_entr_for_del, conn)
        query_bal_for_del = """select * from _3psold_balance where company_id=%s and date='%s'"""%(company_id, date_str)
        df_bal_for_del = pd.read_sql(query_bal_for_del, conn)
        query_ship_for_del = """select * from _3psold_shipment where company_id=%s and date='%s'"""%(company_id, date_str)
        df_ship_for_del = pd.read_sql(query_ship_for_del, conn)

        if (((len(df_entr_for_del)>0) | (len(df_bal_for_del)>0) | (len(df_ship_for_del)>0)) & (file_name not in err_files)):
            #print('aaaa')
            delete_query = """delete from _3psold_entrance where company_id=%s and date='%s';
                              delete from _3psold_shipment where company_id=%s and date='%s';
                              delete from _3psold_balance where company_id=%s and date='%s'
                           """%(company_id, date_str, company_id, date_str, company_id, date_str)
            cur.execute(delete_query)
            conn.commit()
            print('Дублирующие данные удалены. Файл - ', file_name, '; за дату - ', date_str)     
            print()

insert_query = insert_query_balance+insert_query_entrance+insert_query_shipment
if len(insert_query) > 0:
    cur.execute(insert_query)
    conn.commit()
    #print(file_name, ' - файл добавлен')
    #print()
    
cur.close()
conn.close()
print(errors) 

C:\Users\mendgaziev\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\mendgaziev\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\mendgaziev\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


3-ПС  САС март 2023.xlsx
2195
Нет поступлений -  3-ПС  САС март 2023.xlsx
28051
 insert into _3psold_shipment values (28052, '353211, Краснодарский край, Динской район, ст.Новотитаровская, ул.Крайняя, 2/11', '62', '400', 'ООО "Газпром-переработка"', '97152834', Null, '40265561000', 'Null', Null, False, '123', '376', '2023-03-01'); 
 insert into _3psold_shipment values (28053, '353211, Краснодарский край, Динской район, ст.Новотитаровская, ул.Крайняя, 2/11', '62', '469', 'АО "Газпромнефть-Московский НПЗ"', '05766623', Null, '45290558000', 'Null', Null, False, '2828.413', '376', '2023-03-01'); 
 insert into _3psold_shipment values (28054, '353211, Краснодарский край, Динской район, ст.Новотитаровская, ул.Крайняя, 2/11', '62', '458', 'АО "Куйбышевский нефтеперерабатывающий завод"', '05766505', Null, '36401378000', 'Null', Null, False, '356.13', '376', '2023-03-01'); 
 insert into _3psold_shipment values (28055, '353211, Краснодарский край, Динской район, ст.Новотитаровская, ул.Крайняя, 2/

C:\Users\mendgaziev\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\mendgaziev\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\mendgaziev\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [31]:
errors

[]